**QC of ETL starting with GDC release 24 clinical tables**

This notebook focuses on the QC of program **MMRF** data_category clinical

This program has a total of five clinical tables present in this release

Tables listed below ---

- `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF`

- `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_diag__treat`

- `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_fam_hist`

- `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_follow`

- `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_follow__mol_test`




##QC table checklist 

Multiple one-to-many tables present QC list

**1. Check schema**

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields?
    
Are the labels correct?

**2. Look at table row number and size**

Do these metrics make sense?

**3. Scroll through table manually**

See if anything stands out - empty columns, etc.

The BigQuery table search user interface is useful in for this test run. The test tier points to the isb-etl-open. 

[ISB-CGC BigQuery table  search test tier](https://isb-cgc-test.appspot.com/bq_meta_search/)

Run a manual check in the console with the steps mentioned in step 1.

*Note from developer:
There are some columns which are sparsely populated (so they might look empty if you’re just scrolling through the table in the GUI), but there should be at least one non-null entry for every column in every table.*

**4. Number of case_id versus BigQuery metadata table**

**5.Check for any duplicate rows present in the table**

**7. Verify case_id count of table against  master rel_clinical_data table**

##Reference material



*   [NextGenETL](https://github.com/isb-cgc/NextGenETL) GitHub repository
*   [ETL QC SOP draft](https://docs.google.com/document/d/1Wskf3BxJLkMjhIXD62B6_TG9h5KRcSp8jSAGqcCP1lQ/edit)

##Before you begin

You need to load the BigQuery module, authenticate ourselves, create a client variable, and load the necessary libraries.


In [1]:
from google.colab import auth
try:
  auth.authenticate_user()
  print('You have been successfully authenticated!')
except:
  print('You have not been authenticated.')

You have been successfully authenticated!


In [2]:
from google.cloud import bigquery
try:
  project_id = 'isb-project-zero' # Update your_project_number with your project number
  client = bigquery.Client(project=project_id)
  print('BigQuery client successfully initialized')
except:
  print('Failed')

BigQuery client successfully initialized


In [3]:
#Install pypika to build a Query 
!pip install pypika
# Import from PyPika
from pypika import Query, Table, Field, Order

import pandas

     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for pypika: filename=PyPika-0.37.7-py2.py3-none-any.whl size=42747 sha256=fc94bedb575e5d75a3ac8c210f081b1939c750bd382c8faaa98be611a8a19777
  Stored in directory: /root/.cache/pip/wheels/40/b2/20/cf67d3c67186b46241b5069c93da2c9beedbb3f08dba75fffe
Successfully built pypika


## READY TO BEGIN TESTING

##Clin MMRF 

**Testing Full ID** `isb-project-zero.GDC_Clinical_Data.rel23_clin_MMRF`

[Table location](https://console.cloud.google.com/bigquery?authuser=1&folder=&organizationId=&project=isb-project-zero&p=isb-project-zero&d=GDC_Clinical_Data&t=rel23_clin_MMRF&page=table)

Source : GDC API

Release version : v24


###test 1 - schema verification

**1. Check schema**

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields
    
Are the labels correct

Google documentation column descriptions for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#column_field_paths_view).

Google documentation table options for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#options_table).

In [32]:
#return all table information for rel24_clin_MMRF

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLES')
clin_query = Query.from_(clin_table) \
                  .select(' table_catalog, table_schema, table_name, table_type ') \
                  .where(clin_table.table_name=='rel24_clin_MMRF') \
                  
clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
clin.head()

,table_catalog,table_schema,table_name,table_type
0,isb-project-zero,GDC_Clinical_Data,rel24_clin_MMRF,BASE TABLE


In [33]:
#return all table information for rel24_clin_MMRF

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_MMRF') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
    print(clin['option_name'][i] + '\n')
    print('\t' + clin['option_value'][i] + '\n')
    print('\t' + clin['option_type'][i] + '\n')

else:

    print('QC of friendly name, table description and labels --- FAILED')

QC of friendly name, table description and labels --- FAILED


In [34]:
#check for empty schemas in dataset rel24_clin_MMRF

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_MMRF') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows
print("Are there any empty cells in the table schema?")
clin.empty

Are there any empty cells in the table schema?


True

FIELD Descriptions pulled example below


In [35]:
#list of field descriptions for table rel24_clin_MMRF

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_MMRF') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
  print(clin['table_name'][i] + '\n')
  print('\t' + clin['column_name'][i] + '\n')
  print('\t' + clin['description'][i] + '\n')

rel24_clin_MMRF

	submitter_id

	

rel24_clin_MMRF

	case_id

	

rel24_clin_MMRF

	diag__treat__count

	Total child record count (located in cases table).

rel24_clin_MMRF

	fam_hist__count

	Total child record count (located in cases table).

rel24_clin_MMRF

	follow__count

	Total child record count (located in cases table).

rel24_clin_MMRF

	primary_site

	

rel24_clin_MMRF

	disease_type

	

rel24_clin_MMRF

	index_date

	

rel24_clin_MMRF

	proj__name

	Display name for the project

rel24_clin_MMRF

	proj__project_id

	

rel24_clin_MMRF

	demo__demographic_id

	

rel24_clin_MMRF

	demo__gender

	Text designations that identify gender. Gender is described as the assemblage of properties that distinguish people on the basis of their societal roles. [Explanatory Comment 1: Identification of gender is based upon self-report and may come from a form, questionnaire, interview, etc.]

rel24_clin_MMRF

	demo__race

	An arbitrary classification of a taxonomic group that is a division of a

In [36]:
# check for empty schemas in dataset rel24_clin_MMRF 

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_MMRF') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows
print("Are there any empty cells in the table schema?")
print(clin)

Are there any empty cells in the table schema?
         table_name  ...                                        description
0   rel24_clin_MMRF  ...                                                   
1   rel24_clin_MMRF  ...                                                   
2   rel24_clin_MMRF  ...  Total child record count (located in cases tab...
3   rel24_clin_MMRF  ...  Total child record count (located in cases tab...
4   rel24_clin_MMRF  ...  Total child record count (located in cases tab...
5   rel24_clin_MMRF  ...                                                   
6   rel24_clin_MMRF  ...                                                   
7   rel24_clin_MMRF  ...                                                   
8   rel24_clin_MMRF  ...                       Display name for the project
9   rel24_clin_MMRF  ...                                                   
10  rel24_clin_MMRF  ...                                                   
11  rel24_clin_MMRF  ...  Text designatio

###test 2 - row number verification

**2. Look at table row number and size**

Do these metrics make sense?

In [9]:
%%bigquery --project isb-project-zero
SELECT COUNT(submitter_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF`

,f0_
0,995


In [10]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel23_clin_MMRF`

,f0_
0,995


In [37]:
%%bigquery --project isb-project-zero
SELECT *
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF`

,submitter_id,case_id,diag__treat__count,fam_hist__count,follow__count,primary_site,disease_type,index_date,proj__name,proj__project_id,demo__demographic_id,demo__gender,demo__race,demo__ethnicity,demo__vital_status,demo__days_to_birth,demo__age_at_index,demo__days_to_death,demo__cause_of_death,demo__state,demo__created_datetime,demo__updated_datetime,diag__diagnosis_id,diag__primary_diagnosis,diag__days_to_last_known_disease_status,diag__progression_or_recurrence,diag__site_of_resection_or_biopsy,diag__age_at_diagnosis,diag__days_to_last_follow_up,diag__tumor_grade,diag__last_known_disease_status,diag__morphology,diag__tumor_stage,diag__iss_stage,diag__tissue_or_organ_of_origin,diag__state,diag__created_datetime,diag__updated_datetime,state,created_datetime,updated_datetime
0,MMRF_1328,e8a89081-0ada-4b93-9bf9-399597ea9d76,64,4,17,Hematopoietic and reticuloendothelial systems,Plasma Cell Tumors,First Treatment,Multiple Myeloma CoMMpass Study,MMRF-COMMPASS,a1785392-3f10-4ecd-b68c-741ea20c8aba,male,white,not hispanic or latino,Alive,-16918.0,46,NaN,None,released,2018-10-08T18:16:45.487734-05:00,2019-06-24T08:07:19.797044-05:00,727d8936-bb2e-40ff-b1e2-1707fd08bcdc,Multiple myeloma,1434.0,unknown,Bone marrow,16918.0,1434.0,Unknown,Unknown tumor status,9732/3,I,I,Bone marrow,released,2018-07-10T14:08:13.021252-05:00,2019-06-24T08:07:19.797044-05:00,released,2018-01-01T02:07:36.073040-06:00,2019-06-24T08:07:19.797044-05:00
1,MMRF_1690,1365c987-1c56-474e-ab2b-7f53cabf3cb0,24,1,14,Hematopoietic and reticuloendothelial systems,Plasma Cell Tumors,First Treatment,Multiple Myeloma CoMMpass Study,MMRF-COMMPASS,661ba2fa-c63a-47d9-bfca-8235689d80b0,female,white,not hispanic or latino,Alive,-21110.0,57,NaN,None,released,2018-10-08T18:16:45.487734-05:00,2019-06-24T08:07:19.797044-05:00,964b4dee-3190-4833-8eb9-3a0712c13f60,Multiple myeloma,1150.0,unknown,Bone marrow,21110.0,1150.0,Unknown,Unknown tumor status,9732/3,I,I,Bone marrow,released,2018-07-10T14:08:13.021252-05:00,2019-06-24T08:07:19.797044-05:00,released,2018-01-01T02:07:36.073040-06:00,2019-06-24T08:07:19.797044-05:00
2,MMRF_2428,6b9d857e-e76b-4901-8959-506801628eaf,24,2,9,Hematopoietic and reticuloendothelial systems,Plasma Cell Tumors,First Treatment,Multiple Myeloma CoMMpass Study,MMRF-COMMPASS,9547c410-33b3-4019-9174-cc7ca61834e6,male,white,not hispanic or latino,Alive,-19783.0,54,NaN,None,released,2018-10-08T18:16:45.487734-05:00,2019-06-24T08:07:19.797044-05:00,25bedab0-effc-49a3-a392-a48394b2a577,Multiple myeloma,715.0,unknown,Bone marrow,19783.0,715.0,Unknown,Unknown tumor status,9732/3,I,I,Bone marrow,released,2018-07-10T14:08:13.021252-05:00,2019-06-24T08:07:19.797044-05:00,released,2018-01-01T02:07:36.073040-06:00,2019-06-24T08:07:19.797044-05:00
3,MMRF_1439,463a106f-de45-4520-8122-aaed112641bb,1,1,3,Hematopoietic and reticuloendothelial systems,Plasma Cell Tumors,First Treatment,Multiple Myeloma CoMMpass Study,MMRF-COMMPASS,0d5af564-1984-4f7f-b349-25cc4a45951d,female,black or african american,not hispanic or latino,Alive,-18235.0,49,NaN,None,released,2018-10-08T18:16:45.487734-05:00,2019-06-24T08:07:19.797044-05:00,d11a62f6-8fb8-48bc-9cfe-e276657ed550,Multiple myeloma,253.0,unknown,Bone marrow,18235.0,253.0,Unknown,Unknown tumor status,9732/3,I,I,Bone marrow,released,2018-07-10T14:08:13.021252-05:00,2019-06-24T08:07:19.797044-05:00,released,2018-01-01T02:07:36.073040-06:00,2019-06-24T08:07:19.797044-05:00
4,MMRF_1033,a3b3a7dd-8559-4b01-8112-d15ec35c5550,2,2,21,Hematopoietic and reticuloendothelial systems,Plasma Cell Tumors,First Treatment,Multiple Myeloma CoMMpass Study,MMRF-COMMPASS,3332c3f4-6d43-47d7-98c6-d7e7c61c7b75,male,asian,not hispanic or latino,Alive,-23692.0,64,NaN,None,released,2018-10-08T18:16:45.487734-05:00,2019-06-24T08:07:19.797044-05:00,6261a985-8a3b-4928-9d48-1d43055c968e,Multiple myeloma,1786.0,unknown,Bone marrow,23692.0,1786.0,Unknown,Unknown tumor status,9732/3,I,I,Bone marrow,released,2018-07-10T14:08:13.021252-05:00,2019-06-24T08:07:19.797044-05:0

###test 3 - manual verification

**3. Scroll through table manually**

See if anything stands out - empty columns, etc.

The BigQuery table search user interface is useful in for this test run. The test tier points to the isb-etl-open. 

ISB-CGC BigQuery table  search [test tier](https://isb-cgc-test.appspot.com/bq_meta_search/).

BigQuery console [isb-project-zero](https://console.cloud.google.com/bigquery?authuser=1&folder=&organizationId=&project=isb-project-zero&p=isb-project-zero&d=GDC_Clinical_Data&t=rel24_clin_MMRF&page=table).

Run a manual check in the console with the steps mentioned in step 1 

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields?
    
Are the labels correct?

*Note from developer:
There are some columns which are sparsely populated (so they might look empty if you’re just scrolling through the table in the GUI), but there should be at least one non-null entry for every column in every table.*

###test 4 - case_gdc_id file metadata table count verification

**4. Number of case_id versus BigQuery metadata table**



In [38]:
# clinical case_id counts table reuslts below

# Query below will display the number of cases presents in this table.

clin_table = Table('`isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF`')
clin_query = Query.from_(clin_table) \
                  .select(' DISTINCT case_id, count(*) as count') \
                  .groupby('case_id')

clin_query_clean = str(clin_query).replace('"', "")
#print(clin_query_clean)
clin = client.query(clin_query_clean).to_dataframe()
print('number of case from submitter_id = ' + str(len(clin.index)))


number of case from submitter_id = 995


In [23]:
# GDC file metadata table case_gdc_id count for clinical below

%%bigquery --project isb-project-zero
SELECT case_gdc_id, program_name
FROM `isb-project-zero.GDC_metadata.rel24_caseData`
where program_name = 'MMRF'
group by case_gdc_id, program_name

,case_gdc_id,program_name
0,a6533b38-fe7d-4c1c-951c-c868e9377af6,MMRF
1,f1b12b03-6135-40a1-9aef-0a600e3b5b9a,MMRF
2,b4f423b6-71b9-459a-a5ed-93208ec27397,MMRF
3,9a2b0af5-bcf5-4398-bcd0-d4f4c7cc9852,MMRF
4,e5992503-81fd-4ba2-acbb-33901a9d498a,MMRF
...,...,...
990,6357d58b-499d-48ac-9195-d42d49fba16c,MMRF
991,9046fcfb-0e05-48fc-a311-d6ef2a422ae1,MMRF
992,56ed99a7-42bb-46a1-acd9-c66733fece55,MMRF
993,de405770-7784-4ca2-991f-2dee14ef221b,MMRF


In [24]:
%%bigquery --project isb-project-zero

SELECT distinct case_id, count(case_id) as count
FROM `isb-project-zero.GDC_metadata.rel24_fileData_current` as active, `isb-project-zero.GDC_Clinical_Data.rel24_clinical_data` as clinical
WHERE program_name = 'MMRF'
AND active.case_gdc_id = clinical.case_id
group by case_id
order by count

,case_id,count
0,9a2b0af5-bcf5-4398-bcd0-d4f4c7cc9852,8
1,1457fce6-b646-4e86-a3a2-38b3081acd0a,8
2,15713a83-8a83-491e-9b6d-52538a3e8478,8
3,af227b27-8550-4ee2-b36f-243842400ae5,8
4,82a77e5c-96f6-43a3-b3c0-7ca35f4983f4,8
...,...,...
990,6357d58b-499d-48ac-9195-d42d49fba16c,74
991,9046fcfb-0e05-48fc-a311-d6ef2a422ae1,74
992,56ed99a7-42bb-46a1-acd9-c66733fece55,82
993,de405770-7784-4ca2-991f-2dee14ef221b,92


###test 5 - duplication verifcation

**5. Check for any duplicate rows present in the table**


In [39]:
%%bigquery --project isb-project-zero

SELECT count(submitter_id) AS count
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF`
GROUP BY submitter_id, case_id, diag__treat__count, fam_hist__count, follow__count, primary_site, disease_type, index_date, demo__demographic_id, demo__gender, demo__race, demo__ethnicity, demo__vital_status, demo__days_to_birth, demo__age_at_index, demo__days_to_death, demo__cause_of_death, demo__state, demo__created_datetime, demo__updated_datetime, diag__diagnosis_id, diag__primary_diagnosis, diag__days_to_last_known_disease_status, diag__progression_or_recurrence, diag__site_of_resection_or_biopsy, diag__age_at_diagnosis, diag__days_to_last_follow_up, diag__tumor_grade, diag__last_known_disease_status, diag__morphology, diag__tumor_stage, diag__iss_stage, diag__tissue_or_organ_of_origin, diag__state, diag__created_datetime, diag__updated_datetime, state, created_datetime, updated_datetime
ORDER BY count DESC
LIMIT 10

,count
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


###test 6 - case_id master clinical data table count verifcation

**6. Verify case_id count of table against master rel_clinical_data table**

In [20]:
# case_id count from the program MMRF clinical table

%%bigquery --project isb-project-zero

select distinct case_id, count(case_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF` 
group by case_id
order by count

,case_id,count
0,e8a89081-0ada-4b93-9bf9-399597ea9d76,1
1,1365c987-1c56-474e-ab2b-7f53cabf3cb0,1
2,6b9d857e-e76b-4901-8959-506801628eaf,1
3,463a106f-de45-4520-8122-aaed112641bb,1
4,a3b3a7dd-8559-4b01-8112-d15ec35c5550,1
...,...,...
990,c93b9815-3f76-48c6-8b6c-1ec2ee08825d,1
991,ddb370db-7348-49a2-bcb0-5afdefa6d644,1
992,851f305b-e03c-4648-9400-facaa7123cc6,1
993,2ff5c839-e056-4fcc-a4a5-43f11535024e,1


In [21]:
# case_id count from the master clinical table

%%bigquery --project isb-project-zero

SELECT distinct case_id, count(case_id) as count
FROM `isb-project-zero.GDC_metadata.rel24_fileData_current` as active, `isb-project-zero.GDC_Clinical_Data.rel24_clinical_data` as clinical
WHERE program_name = 'MMRF'
AND active.case_gdc_id = clinical.case_id
group by case_id
order by count


,case_id,count
0,9a2b0af5-bcf5-4398-bcd0-d4f4c7cc9852,8
1,1457fce6-b646-4e86-a3a2-38b3081acd0a,8
2,15713a83-8a83-491e-9b6d-52538a3e8478,8
3,af227b27-8550-4ee2-b36f-243842400ae5,8
4,82a77e5c-96f6-43a3-b3c0-7ca35f4983f4,8
...,...,...
990,6357d58b-499d-48ac-9195-d42d49fba16c,74
991,9046fcfb-0e05-48fc-a311-d6ef2a422ae1,74
992,56ed99a7-42bb-46a1-acd9-c66733fece55,82
993,de405770-7784-4ca2-991f-2dee14ef221b,92


##Clin MMRF_diag__treat

**Testing Full ID** `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_diag__treat`

[Table location](https://console.cloud.google.com/bigquery?authuser=1&folder=&organizationId=&project=isb-project-zero&p=isb-project-zero&d=GDC_Clinical_Data&t=rel24_clin_MMRF_diag__treat&page=table)

Source : GDC API

Release version : v24


###test 1 - schema verification

**1. Check schema**

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields
    
Are the labels correct

Google documentation column descriptions for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#column_field_paths_view).

Google documentation table options for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#options_table).

In [28]:
#return all table information for rel24_clin_MMRF_diag__treat

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLES')
clin_query = Query.from_(clin_table) \
                  .select(' table_catalog, table_schema, table_name, table_type ') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_diag__treat') \
                  
clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
clin.head()

,table_catalog,table_schema,table_name,table_type
0,isb-project-zero,GDC_Clinical_Data,rel24_clin_MMRF_diag__treat,BASE TABLE


In [27]:
#return all table information for rel24_clin_MMRF_diag__treat

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_diag__treat') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
    print(clin['option_name'][i] + '\n')
    print('\t' + clin['option_value'][i] + '\n')
    print('\t' + clin['option_type'][i] + '\n')

else:

    print('QC of friendly name, table description and labels --- FAILED')

QC of friendly name, table description and labels --- FAILED


In [50]:
#check for empty schemas in dataset rel24_clin_MMRF_diag__treat

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_diag__treat') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows
print("Are there any empty cells in the table schema?")
clin.empty

Are there any empty cells in the table schema?


True

FIELD Descriptions pulled example below


In [30]:
#list of field descriptions for table rel24_clin_MMRF_diag__treat

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_diag__treat') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
  print(clin['table_name'][i] + '\n')
  print('\t' + clin['column_name'][i] + '\n')
  print('\t' + clin['description'][i] + '\n')

rel24_clin_MMRF_diag__treat

	diag__treat__treatment_id

	

rel24_clin_MMRF_diag__treat

	diag__diagnosis_id

	Reference to ancestor diag__diagnosis_id, located in rel24_clin_MMRF_diag.

rel24_clin_MMRF_diag__treat

	case_id

	Reference to ancestor case_id, located in rel24_clin_MMRF.

rel24_clin_MMRF_diag__treat

	diag__treat__days_to_treatment_start

	Number of days between the date used for index and the date the treatment started.

rel24_clin_MMRF_diag__treat

	diag__treat__treatment_type

	Text term that describes the kind of treatment administered.

rel24_clin_MMRF_diag__treat

	diag__treat__treatment_or_therapy

	A yes/no/unknown/not applicable indicator related to the administration of therapeutic agents received.

rel24_clin_MMRF_diag__treat

	diag__treat__therapeutic_agents

	Text identification of the individual agent(s) used as part of a treatment regimen.

rel24_clin_MMRF_diag__treat

	diag__treat__days_to_treatment_end

	Number of days between the date used for index and 

In [31]:
# check for empty schemas in dataset rel24_clin_MMRF_diag__treat

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_diag__treat') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
print(clin)

                     table_name  ...                                        description
0   rel24_clin_MMRF_diag__treat  ...                                                   
1   rel24_clin_MMRF_diag__treat  ...  Reference to ancestor diag__diagnosis_id, loca...
2   rel24_clin_MMRF_diag__treat  ...  Reference to ancestor case_id, located in rel2...
3   rel24_clin_MMRF_diag__treat  ...  Number of days between the date used for index...
4   rel24_clin_MMRF_diag__treat  ...  Text term that describes the kind of treatment...
5   rel24_clin_MMRF_diag__treat  ...  A yes/no/unknown/not applicable indicator rela...
6   rel24_clin_MMRF_diag__treat  ...  Text identification of the individual agent(s)...
7   rel24_clin_MMRF_diag__treat  ...  Number of days between the date used for index...
8   rel24_clin_MMRF_diag__treat  ...  The text term used to describe the regimen or ...
9   rel24_clin_MMRF_diag__treat  ...                   The current state of the object.
10  rel24_clin_MMRF_diag__treat 

###test 2 - row number verification

**2. Look at table row number and size**

Do these metrics make sense?

In [15]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_diag__treat`

,f0_
0,7184


In [16]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel23_clin_MMRF_diag__treat`

,f0_
0,7184


In [40]:
%%bigquery --project isb-project-zero
SELECT *
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_diag__treat`

,diag__treat__treatment_id,diag__diagnosis_id,case_id,diag__treat__days_to_treatment_start,diag__treat__treatment_type,diag__treat__treatment_or_therapy,diag__treat__therapeutic_agents,diag__treat__days_to_treatment_end,diag__treat__regimen_or_line_of_therapy,diag__treat__state,diag__treat__created_datetime,diag__treat__updated_datetime
0,3601a62f-cd5a-465a-94f6-1341dd16a5f3,37a32ab7-f023-4ef0-bab3-42815ca132ef,089a1abc-aa81-46a2-b02f-f0f78a1b3d84,1536.0,None,yes,Cyclophosphamide,1578.0,Fifth line of therapy,released,2018-07-10T15:03:25.257473-05:00,2019-06-24T08:07:19.797044-05:00
1,fcdcdd36-e6e8-4c7d-87d0-f35d28245447,37a32ab7-f023-4ef0-bab3-42815ca132ef,089a1abc-aa81-46a2-b02f-f0f78a1b3d84,1564.0,None,yes,Pomalidomide,1578.0,Fifth line of therapy,released,2018-07-10T15:03:25.257473-05:00,2019-06-24T08:07:19.797044-05:00
2,163b9e57-75e7-4058-8a35-0ae5eef89fe5,37a32ab7-f023-4ef0-bab3-42815ca132ef,089a1abc-aa81-46a2-b02f-f0f78a1b3d84,1449.0,None,yes,Thalidomide,1554.0,Fifth line of therapy,released,2018-07-10T15:03:25.257473-05:00,2019-06-24T08:07:19.797044-05:00
3,99f903f5-9992-49a0-a182-11648ab8b42b,37a32ab7-f023-4ef0-bab3-42815ca132ef,089a1abc-aa81-46a2-b02f-f0f78a1b3d84,1449.0,None,yes,Dexamethasone,1578.0,Fifth line of therapy,released,2018-07-10T15:03:25.257473-05:00,2019-06-24T08:07:19.797044-05:00
4,34ac51ff-0e0c-4f0e-ad38-e56336210254,37a32ab7-f023-4ef0-bab3-42815ca132ef,089a1abc-aa81-46a2-b02f-f0f78a1b3d84,1449.0,None,yes,Daratumumab,1578.0,Fifth line of therapy,released,2018-07-10T15:03:25.257473-05:00,2019-06-24T08:07:19.797044-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...
7179,b943151e-162d-4999-aaf4-d886e88bd195,56e3ba7f-614b-4a01-90ae-72e2aa7f022f,273a1d4c-807a-4470-936d-e76e1e2ae190,775.0,"Stem Cell Transplantation, Autologous",yes,None,775.0,Second line of therapy,released,2018-07-10T15:03:25.257473-05:00,2019-06-24T08:07:19.797044-05:00
7180,1d0534b5-1e83-4158-9602-7bdc5c044c3a,9e50bb78-0946-48e9-b36d-a0b191f8c547,3bfb8a82-4b5f-49b6-bb27-5b756b1f3eed,247.0,"Stem Cell Transplantation, Autologous",yes,None,247.0,Second line of therapy,released,2018-07-10T15:03:25.257473-05:00,2019-06-24T08:07:19.797044-05:00
7181,20a02517-69c0-4824-859d-8f2dc6d43932,8fa9457a-1b58-49c7-9c05-c3b2ca61b4ce,2f5e648f-7cc3-44db-8c84-a7b8ffcccf97,336.0,"Stem Cell Transplantation, Autologous",yes,None,336.0,Second line of therapy,released,2018-07-10T15:03:25.257473-05:00,2019-06-24T08:07:19.797044-05:00
7182,f393bb15-6d62-4751-9164-0a28d21ad504,1b809cce-6daa-4bab-b7d3-c9685df21f7a,ba2be3d7-4305-4dc3-a912-cf85440492e0,1215.0,"Stem Cell Transplantation, Autologous",yes,None,1215.0,Second line of therapy,released,2018-07-10T15:03:25.257473-05:00,2019-06-24T08:07:19.797044-05:00


###test 3 - manual verification

**3. Scroll through table manually**

See if anything stands out - empty columns, etc.

The BigQuery table search user interface is useful in for this test run. The test tier points to the isb-etl-open. 

ISB-CGC BigQuery table  search [test tier](https://isb-cgc-test.appspot.com/bq_meta_search/).

BigQuery console [isb-project-zero](https://console.cloud.google.com/bigquery?authuser=1&folder=&organizationId=&project=isb-project-zero&p=isb-project-zero&d=GDC_Clinical_Data&t=rel24_clin_MMRF_diag__treat&page=table).

Run a manual check in the console with the steps mentioned in step 1.

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields?
    
Are the labels correct?

###test 4 - case_gdc_id file metadata table count verification

**4. Number of case_id versus BigQuery metadata table**



In [41]:
# clinical case_id counts table reuslts below

# Query below will display the number of cases presents in this table.

clin_table = Table('`isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_diag__treat`')
clin_query = Query.from_(clin_table) \
                  .select(' DISTINCT case_id, count(*) as count') \
                  .groupby('case_id')

clin_query_clean = str(clin_query).replace('"', "")
#print(clin_query_clean)
clin = client.query(clin_query_clean).to_dataframe()
print('number of case from submitter_id = ' + str(len(clin.index)))


number of case from submitter_id = 994


In [42]:
# GDC file metadata table case_gdc_id count for clinical below

%%bigquery --project isb-project-zero
SELECT case_gdc_id, program_name
FROM `isb-project-zero.GDC_metadata.rel24_caseData`
where program_name = 'MMRF'
group by case_gdc_id, program_name

,case_gdc_id,program_name
0,a6533b38-fe7d-4c1c-951c-c868e9377af6,MMRF
1,f1b12b03-6135-40a1-9aef-0a600e3b5b9a,MMRF
2,b4f423b6-71b9-459a-a5ed-93208ec27397,MMRF
3,9a2b0af5-bcf5-4398-bcd0-d4f4c7cc9852,MMRF
4,e5992503-81fd-4ba2-acbb-33901a9d498a,MMRF
...,...,...
990,6357d58b-499d-48ac-9195-d42d49fba16c,MMRF
991,9046fcfb-0e05-48fc-a311-d6ef2a422ae1,MMRF
992,56ed99a7-42bb-46a1-acd9-c66733fece55,MMRF
993,de405770-7784-4ca2-991f-2dee14ef221b,MMRF


In [45]:
%%bigquery --project isb-project-zero

SELECT distinct case_id, count(case_id) as count
FROM `isb-project-zero.GDC_metadata.rel24_caseData` as active, `isb-project-zero.GDC_Clinical_Data.rel24_clinical_data` as clinical
WHERE program_name = 'MMRF'
AND active.case_gdc_id = clinical.case_id
group by case_id
order by count

,case_id,count
0,a6533b38-fe7d-4c1c-951c-c868e9377af6,1
1,f1b12b03-6135-40a1-9aef-0a600e3b5b9a,1
2,b4f423b6-71b9-459a-a5ed-93208ec27397,1
3,9a2b0af5-bcf5-4398-bcd0-d4f4c7cc9852,1
4,e5992503-81fd-4ba2-acbb-33901a9d498a,1
...,...,...
990,6357d58b-499d-48ac-9195-d42d49fba16c,1
991,9046fcfb-0e05-48fc-a311-d6ef2a422ae1,1
992,56ed99a7-42bb-46a1-acd9-c66733fece55,1
993,de405770-7784-4ca2-991f-2dee14ef221b,1


###test 5 - duplication verifcation

**5. Check for any duplicate rows present in the table**

In [47]:
%%bigquery --project isb-project-zero

SELECT count(case_id) AS count
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_diag__treat`
group by diag__treat__treatment_id, diag__diagnosis_id, case_id, diag__treat__days_to_treatment_start, diag__treat__treatment_type, diag__treat__treatment_or_therapy, diag__treat__therapeutic_agents, diag__treat__days_to_treatment_end, diag__treat__regimen_or_line_of_therapy, diag__treat__state, diag__treat__created_datetime, diag__treat__updated_datetime
ORDER BY count DESC
LIMIT 10

,count
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


###test 6 - case_id master clinical data table count verifcation

**6. Verify case_id count of table against master rel_clinical_data table**

In [25]:
# case_id count from the program MMRF clinical table

%%bigquery --project isb-project-zero

select distinct case_id, count(case_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF` 
group by case_id
order by count

,case_id,count
0,e8a89081-0ada-4b93-9bf9-399597ea9d76,1
1,1365c987-1c56-474e-ab2b-7f53cabf3cb0,1
2,6b9d857e-e76b-4901-8959-506801628eaf,1
3,463a106f-de45-4520-8122-aaed112641bb,1
4,a3b3a7dd-8559-4b01-8112-d15ec35c5550,1
...,...,...
990,c93b9815-3f76-48c6-8b6c-1ec2ee08825d,1
991,ddb370db-7348-49a2-bcb0-5afdefa6d644,1
992,851f305b-e03c-4648-9400-facaa7123cc6,1
993,2ff5c839-e056-4fcc-a4a5-43f11535024e,1


In [26]:
# case_id count from the program MMRF_diag__treat clinical table

%%bigquery --project isb-project-zero

select distinct case_id, count(case_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_diag__treat` 
group by case_id
order by count

,case_id,count
0,6708920e-ecd2-4682-8a33-52a64b710f03,1
1,85e1a95c-030d-4e6a-ad2c-e468de783b41,1
2,463a106f-de45-4520-8122-aaed112641bb,1
3,8db0ed0d-03bb-4bc6-917d-4bd861472ed1,1
4,ce2364f6-fac4-43f2-9947-8c9d11c294a1,1
...,...,...
989,6ce3b375-59a3-49fc-9d6f-1670450d4cf1,26
990,f65c5bfa-cf3b-4c03-a4b4-47c8d935960b,28
991,c9d10f0c-9e61-41c5-8a86-340f8411bec6,29
992,af227b27-8550-4ee2-b36f-243842400ae5,32


##Clin MMRF_fam_hist

**Testing Full ID** `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_fam_hist`

[Table location](https://console.cloud.google.com/bigquery?project=high-transit-276919&authuser=2&p=isb-project-zero&d=GDC_metadata&t=rel24_fam_histpage=table)

Source : GDC API

Release version : v24

###test 1 - schema verification

**1. Check schema**

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields
    
Are the labels correct

Google documentation column descriptions for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#column_field_paths_view).

Google documentation table options for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#options_table).

In [48]:
#return all table information for rel24_clin_MMRF_fam_hist

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLES')
clin_query = Query.from_(clin_table) \
                  .select(' table_catalog, table_schema, table_name, table_type ') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_fam_hist') \
                  
clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
clin.head()

,table_catalog,table_schema,table_name,table_type
0,isb-project-zero,GDC_Clinical_Data,rel24_clin_MMRF_fam_hist,BASE TABLE


In [49]:
#return all table information for rel24_clin_MMRF_fam_hist

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_fam_hist') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
    print(clin['option_name'][i] + '\n')
    print('\t' + clin['option_value'][i] + '\n')
    print('\t' + clin['option_type'][i] + '\n')

else:

    print('QC of friendly name, table description and labels --- FAILED')

QC of friendly name, table description and labels --- FAILED


In [51]:
#check for empty schemas in dataset rel24_clin_MMRF_fam_hist

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_fam_hist') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows
print("Are there any empty cells in the table schema?")
clin.empty

Are there any empty cells in the table schema?


True

FIELD Descriptions pulled example below

In [52]:
#list of field descriptions for table rel24_clin_MMRF_fam_hist

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_fam_hist') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
  print(clin['table_name'][i] + '\n')
  print('\t' + clin['column_name'][i] + '\n')
  print('\t' + clin['description'][i] + '\n')

rel24_clin_MMRF_fam_hist

	fam_hist__family_history_id

	

rel24_clin_MMRF_fam_hist

	case_id

	Reference to ancestor case_id, located in rel24_clin_MMRF.

rel24_clin_MMRF_fam_hist

	fam_hist__relative_with_cancer_history

	The yes/no/unknown indicator used to describe whether any of the patient's relatives have a history of cancer.

rel24_clin_MMRF_fam_hist

	fam_hist__relationship_primary_diagnosis

	The text term used to describe the malignant diagnosis of the patient's relative with a history of cancer.

rel24_clin_MMRF_fam_hist

	fam_hist__relationship_type

	The subgroup that describes the state of connectedness between members of the unit of society organized around kinship ties.

rel24_clin_MMRF_fam_hist

	fam_hist__relationship_gender

	The text term used to describe the gender of the patient's relative with a history of cancer.

rel24_clin_MMRF_fam_hist

	fam_hist__state

	The current state of the object.

rel24_clin_MMRF_fam_hist

	fam_hist__created_datetime

	A combination 

In [53]:
# check for empty schemas in dataset rel24_clin_MMRF_fam_hist

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_fam_hist') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
print(clin)

                 table_name  ...                                        description
0  rel24_clin_MMRF_fam_hist  ...                                                   
1  rel24_clin_MMRF_fam_hist  ...  Reference to ancestor case_id, located in rel2...
2  rel24_clin_MMRF_fam_hist  ...  The yes/no/unknown indicator used to describe ...
3  rel24_clin_MMRF_fam_hist  ...  The text term used to describe the malignant d...
4  rel24_clin_MMRF_fam_hist  ...  The subgroup that describes the state of conne...
5  rel24_clin_MMRF_fam_hist  ...  The text term used to describe the gender of t...
6  rel24_clin_MMRF_fam_hist  ...                   The current state of the object.
7  rel24_clin_MMRF_fam_hist  ...  A combination of date and time of day in the f...
8  rel24_clin_MMRF_fam_hist  ...  A combination of date and time of day in the f...

[9 rows x 3 columns]


###test 2 - row number verification

**2. Look at table row number and size**

Do these metrics make sense?

In [54]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_fam_hist`

,f0_
0,1121


In [55]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel23_clin_MMRF_fam_hist`

,f0_
0,1121


In [56]:
%%bigquery --project isb-project-zero
SELECT *
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_fam_hist`

,fam_hist__family_history_id,case_id,fam_hist__relative_with_cancer_history,fam_hist__relationship_primary_diagnosis,fam_hist__relationship_type,fam_hist__relationship_gender,fam_hist__state,fam_hist__created_datetime,fam_hist__updated_datetime
0,d80087ce-ca49-48c7-b05e-66d4dfc34b65,f65c5bfa-cf3b-4c03-a4b4-47c8d935960b,no,None,None,None,released,2018-06-28T15:38:11.378107-05:00,2019-06-24T08:07:19.797044-05:00
1,2bdcb3f4-e093-44e6-82c8-408716156c61,1365c987-1c56-474e-ab2b-7f53cabf3cb0,no,None,None,None,released,2018-06-28T15:38:11.378107-05:00,2019-06-24T08:07:19.797044-05:00
2,5324b40c-3739-48d7-976c-979df5d11f17,af227b27-8550-4ee2-b36f-243842400ae5,no,None,None,None,released,2018-06-28T15:38:11.378107-05:00,2019-06-24T08:07:19.797044-05:00
3,1bd86050-84b6-4253-b9d6-22f11d118a27,0818bdf7-9c1a-413f-873a-3d6d8fbbe6d7,no,None,None,None,released,2018-06-28T15:38:11.378107-05:00,2019-06-24T08:07:19.797044-05:00
4,c18bc2df-64fd-4fa5-b154-6c1b64671665,6ce3b375-59a3-49fc-9d6f-1670450d4cf1,unknown,None,None,None,released,2018-06-28T15:38:11.378107-05:00,2019-06-24T08:07:19.797044-05:00
...,...,...,...,...,...,...,...,...,...
1116,178031eb-ac07-4393-8eec-aef6489e11e5,113abaaf-3eb2-4318-a5a1-d0564d7b942f,yes,Colorectal Cancer,Unknown,unknown,released,2018-06-28T15:38:11.378107-05:00,2019-06-24T08:07:19.797044-05:00
1117,fa5f4e98-032b-4afb-a665-8e5a7302d357,b5c025c8-845f-4952-9eb5-71e979b3f35e,yes,Colorectal Cancer,Maternal Grandmother,female,released,2018-06-28T15:38:11.378107-05:00,2019-06-24T08:07:19.797044-05:00
1118,6d98407f-0d41-4a10-9393-4979a2a85a4e,0ed5c242-5432-462e-9fcb-13e4b6460840,yes,Colorectal Cancer,Maternal Grandmother,female,released,2018-06-28T15:38:11.378107-05:00,2019-06-24T08:07:19.797044-05:00
1119,ed4ce3b0-23a1-4d7b-84ad-6e678a8b2dec,85ac98f4-1e36-4340-95ab-24132db58d43,yes,Colorectal Cancer,Maternal Grandmother,female,released,2018-06-28T15:38:11.378107-05:00,2019-06-24T08:07:19.797044-05:00


###test 3 - manual verification

**3. Scroll through table manually**

See if anything stands out - empty columns, etc.

The BigQuery table search user interface is useful in for this test run. The test tier points to the isb-etl-open. 

ISB-CGC BigQuery table  search [test tier](https://isb-cgc-test.appspot.com/bq_meta_search/).

BigQuery console [isb-project-zero](https://console.cloud.google.com/bigquery?project=high-transit-276919&authuser=2&p=isb-project-zero&d=GDC_metadata&t=rel24_clin_MMRF_fam_hist&page=table).

Run a manual check in the console with the steps mentioned in step 1 

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields?
    
Are the labels correct?

###test 4 - case_gdc_id file metadata table count verification

**4. Number of case_id versus BigQuery metadata table**

In [57]:
# clinical case_id counts table reuslts below

# Query below will display the number of cases presents in this table.

clin_table = Table('`isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_fam_hist`')
clin_query = Query.from_(clin_table) \
                  .select(' DISTINCT case_id, count(*) as count') \
                  .groupby('case_id')

clin_query_clean = str(clin_query).replace('"', "")
#print(clin_query_clean)
clin = client.query(clin_query_clean).to_dataframe()
print('number of case from submitter_id = ' + str(len(clin.index)))

number of case from submitter_id = 826


In [58]:
# GDC file metadata table case_gdc_id count for clinical below

%%bigquery --project isb-project-zero
SELECT case_gdc_id, program_name
FROM `isb-project-zero.GDC_metadata.rel24_caseData`
where program_name = 'MMRF'
group by case_gdc_id, program_name

,case_gdc_id,program_name
0,a6533b38-fe7d-4c1c-951c-c868e9377af6,MMRF
1,f1b12b03-6135-40a1-9aef-0a600e3b5b9a,MMRF
2,b4f423b6-71b9-459a-a5ed-93208ec27397,MMRF
3,9a2b0af5-bcf5-4398-bcd0-d4f4c7cc9852,MMRF
4,e5992503-81fd-4ba2-acbb-33901a9d498a,MMRF
...,...,...
990,6357d58b-499d-48ac-9195-d42d49fba16c,MMRF
991,9046fcfb-0e05-48fc-a311-d6ef2a422ae1,MMRF
992,56ed99a7-42bb-46a1-acd9-c66733fece55,MMRF
993,de405770-7784-4ca2-991f-2dee14ef221b,MMRF


###test 5 - duplication verifcation

**5. Check for any duplicate rows present in the table**

In [59]:
%%bigquery --project isb-project-zero

SELECT count(case_id) AS count
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_fam_hist`
group by fam_hist__family_history_id, case_id, fam_hist__relative_with_cancer_history, fam_hist__relationship_primary_diagnosis, fam_hist__relationship_type, fam_hist__relationship_gender, fam_hist__state, fam_hist__created_datetime, fam_hist__updated_datetime
ORDER BY count DESC
LIMIT 10

,count
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


###test 6 - case_id master clinical data table count verifcation

**6. Verify case_id count of table against master rel_clinical_data table**

In [60]:
# case_id count from the program MMRF clinical table

%%bigquery --project isb-project-zero

select distinct case_id, count(case_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF` 
group by case_id
order by count

,case_id,count
0,e8a89081-0ada-4b93-9bf9-399597ea9d76,1
1,1365c987-1c56-474e-ab2b-7f53cabf3cb0,1
2,6b9d857e-e76b-4901-8959-506801628eaf,1
3,463a106f-de45-4520-8122-aaed112641bb,1
4,a3b3a7dd-8559-4b01-8112-d15ec35c5550,1
...,...,...
990,c93b9815-3f76-48c6-8b6c-1ec2ee08825d,1
991,ddb370db-7348-49a2-bcb0-5afdefa6d644,1
992,851f305b-e03c-4648-9400-facaa7123cc6,1
993,2ff5c839-e056-4fcc-a4a5-43f11535024e,1


In [61]:
# case_id count from the program MMRF_fam_hist clinical table

%%bigquery --project isb-project-zero

select distinct case_id, count(case_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_fam_hist` 
group by case_id
order by count

,case_id,count
0,f65c5bfa-cf3b-4c03-a4b4-47c8d935960b,1
1,1365c987-1c56-474e-ab2b-7f53cabf3cb0,1
2,af227b27-8550-4ee2-b36f-243842400ae5,1
3,0818bdf7-9c1a-413f-873a-3d6d8fbbe6d7,1
4,6ce3b375-59a3-49fc-9d6f-1670450d4cf1,1
...,...,...
821,5461b1f9-b4d0-4f76-97c8-f8ea3940f29a,5
822,2d13b839-d24a-4fb4-ba7b-d357e0237ca9,5
823,cfce9dc2-683b-46fc-bbf5-0e46bc20b2ae,5
824,54321e01-7416-4d3f-ba95-904b40c8a97d,7


##Clin MMRF_follow

**Testing Full ID** `isb-project-zero.GDC_Clinical_Data.rel23_clin_MMRF_follow`

[Table location](https://console.cloud.google.com/bigquery?project=high-transit-276919&authuser=2&p=isb-project-zero&d=GDC_metadata&t=rel23_fileData_legacy&page=table)

Source : GDC API

Release version : v24

###test 1 - schema verification

**1. Check schema**

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields
    
Are the labels correct

Google documentation column descriptions for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#column_field_paths_view).

Google documentation table options for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#options_table).

In [62]:
#return all table information for rel24_clin_MMRF_follow

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLES')
clin_query = Query.from_(clin_table) \
                  .select(' table_catalog, table_schema, table_name, table_type ') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_follow') \
                  
clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
clin.head()

,table_catalog,table_schema,table_name,table_type
0,isb-project-zero,GDC_Clinical_Data,rel24_clin_MMRF_follow,BASE TABLE


In [66]:
#return all table information for rel24_clin_MMRF_follow
clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_follow') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
    print(clin['option_name'][i] + '\n')
    print('\t' + clin['option_value'][i] + '\n')
    print('\t' + clin['option_type'][i] + '\n')

else:

    print('QC of friendly name, table description and labels --- FAILED')

QC of friendly name, table description and labels --- FAILED


In [67]:
#check for empty schemas in dataset rel24_clin_MMRF_follow

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_follow') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows
print("Are there any empty cells in the table schema?")
clin.empty

Are there any empty cells in the table schema?


True

FIELD Descriptions pulled example below




In [68]:
#list of field descriptions for table rel24_clin_MMRF_follow

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_follow') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
  print(clin['table_name'][i] + '\n')
  print('\t' + clin['column_name'][i] + '\n')
  print('\t' + clin['description'][i] + '\n')

rel24_clin_MMRF_follow

	follow__follow_up_id

	Reference to ancestor follow__follow_up_id, located in rel24_clin_MMRF_follow.

rel24_clin_MMRF_follow

	case_id

	Reference to ancestor case_id, located in rel24_clin_MMRF.

rel24_clin_MMRF_follow

	follow__mol_test__count

	Total child record count (located in cases.follow_ups table).

rel24_clin_MMRF_follow

	follow__days_to_follow_up

	Number of days between the date used for index and the date of the patient's last follow-up appointment or contact.

rel24_clin_MMRF_follow

	follow__height

	The height of the patient in centimeters.

rel24_clin_MMRF_follow

	follow__weight

	The weight of the patient measured in kilograms.

rel24_clin_MMRF_follow

	follow__ecog_performance_status

	The ECOG functional performance status of the patient/participant.

rel24_clin_MMRF_follow

	follow__state

	The current state of the object.

rel24_clin_MMRF_follow

	follow__created_datetime

	A combination of date and time of day in the form [-]CCYY-MM-D

In [69]:
# check for empty schemas in dataset rel24_clin_MMRF_follow
clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_follow') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
print(clin)

               table_name  ...                                        description
0  rel24_clin_MMRF_follow  ...  Reference to ancestor follow__follow_up_id, lo...
1  rel24_clin_MMRF_follow  ...  Reference to ancestor case_id, located in rel2...
2  rel24_clin_MMRF_follow  ...  Total child record count (located in cases.fol...
3  rel24_clin_MMRF_follow  ...  Number of days between the date used for index...
4  rel24_clin_MMRF_follow  ...          The height of the patient in centimeters.
5  rel24_clin_MMRF_follow  ...   The weight of the patient measured in kilograms.
6  rel24_clin_MMRF_follow  ...  The ECOG functional performance status of the ...
7  rel24_clin_MMRF_follow  ...                   The current state of the object.
8  rel24_clin_MMRF_follow  ...  A combination of date and time of day in the f...
9  rel24_clin_MMRF_follow  ...  A combination of date and time of day in the f...

[10 rows x 3 columns]


###test 2 - row number verification

**2. Look at table row number and size**

Do these metrics make sense?

In [70]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_follow`

,f0_
0,9049


In [71]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel23_clin_MMRF_follow`

,f0_
0,9049


In [72]:
%%bigquery --project isb-project-zero
SELECT *
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_fam_hist`

,fam_hist__family_history_id,case_id,fam_hist__relative_with_cancer_history,fam_hist__relationship_primary_diagnosis,fam_hist__relationship_type,fam_hist__relationship_gender,fam_hist__state,fam_hist__created_datetime,fam_hist__updated_datetime
0,d80087ce-ca49-48c7-b05e-66d4dfc34b65,f65c5bfa-cf3b-4c03-a4b4-47c8d935960b,no,None,None,None,released,2018-06-28T15:38:11.378107-05:00,2019-06-24T08:07:19.797044-05:00
1,2bdcb3f4-e093-44e6-82c8-408716156c61,1365c987-1c56-474e-ab2b-7f53cabf3cb0,no,None,None,None,released,2018-06-28T15:38:11.378107-05:00,2019-06-24T08:07:19.797044-05:00
2,5324b40c-3739-48d7-976c-979df5d11f17,af227b27-8550-4ee2-b36f-243842400ae5,no,None,None,None,released,2018-06-28T15:38:11.378107-05:00,2019-06-24T08:07:19.797044-05:00
3,1bd86050-84b6-4253-b9d6-22f11d118a27,0818bdf7-9c1a-413f-873a-3d6d8fbbe6d7,no,None,None,None,released,2018-06-28T15:38:11.378107-05:00,2019-06-24T08:07:19.797044-05:00
4,c18bc2df-64fd-4fa5-b154-6c1b64671665,6ce3b375-59a3-49fc-9d6f-1670450d4cf1,unknown,None,None,None,released,2018-06-28T15:38:11.378107-05:00,2019-06-24T08:07:19.797044-05:00
...,...,...,...,...,...,...,...,...,...
1116,178031eb-ac07-4393-8eec-aef6489e11e5,113abaaf-3eb2-4318-a5a1-d0564d7b942f,yes,Colorectal Cancer,Unknown,unknown,released,2018-06-28T15:38:11.378107-05:00,2019-06-24T08:07:19.797044-05:00
1117,fa5f4e98-032b-4afb-a665-8e5a7302d357,b5c025c8-845f-4952-9eb5-71e979b3f35e,yes,Colorectal Cancer,Maternal Grandmother,female,released,2018-06-28T15:38:11.378107-05:00,2019-06-24T08:07:19.797044-05:00
1118,6d98407f-0d41-4a10-9393-4979a2a85a4e,0ed5c242-5432-462e-9fcb-13e4b6460840,yes,Colorectal Cancer,Maternal Grandmother,female,released,2018-06-28T15:38:11.378107-05:00,2019-06-24T08:07:19.797044-05:00
1119,ed4ce3b0-23a1-4d7b-84ad-6e678a8b2dec,85ac98f4-1e36-4340-95ab-24132db58d43,yes,Colorectal Cancer,Maternal Grandmother,female,released,2018-06-28T15:38:11.378107-05:00,2019-06-24T08:07:19.797044-05:00


###test 3 - manual verification

**3. Scroll through table manually**

See if anything stands out - empty columns, etc.

The BigQuery table search user interface is useful in for this test run. The test tier points to the isb-etl-open. 

ISB-CGC BigQuery table  search [test tier](https://isb-cgc-test.appspot.com/bq_meta_search/).

BigQuery console [isb-project-zero](https://console.cloud.google.com/bigquery?project=high-transit-276919&authuser=2&p=isb-project-zero&d=GDC_metadata&t=rel24_clin_MMRF_fam_hist&page=table).

Run a manual check in the console with the steps mentioned in step 1 

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields?
    
Are the labels correct?

###test 4 - case_gdc_id file metadata table count verification

**4. Number of case_id versus BigQuery metadata table**

In [73]:
# clinical case_id counts table reuslts below

# Query below will display the number of cases presents in this table.

clin_table = Table('`isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_follow`')
clin_query = Query.from_(clin_table) \
                  .select(' DISTINCT case_id, count(*) as count') \
                  .groupby('case_id')

clin_query_clean = str(clin_query).replace('"', "")
#print(clin_query_clean)
clin = client.query(clin_query_clean).to_dataframe()
print('number of case from submitter_id = ' + str(len(clin.index)))

number of case from submitter_id = 995


In [74]:
# GDC file metadata table case_gdc_id count for clinical below

%%bigquery --project isb-project-zero
SELECT case_gdc_id, program_name
FROM `isb-project-zero.GDC_metadata.rel24_caseData`
where program_name = 'MMRF'
group by case_gdc_id, program_name

,case_gdc_id,program_name
0,a6533b38-fe7d-4c1c-951c-c868e9377af6,MMRF
1,f1b12b03-6135-40a1-9aef-0a600e3b5b9a,MMRF
2,b4f423b6-71b9-459a-a5ed-93208ec27397,MMRF
3,9a2b0af5-bcf5-4398-bcd0-d4f4c7cc9852,MMRF
4,e5992503-81fd-4ba2-acbb-33901a9d498a,MMRF
...,...,...
990,6357d58b-499d-48ac-9195-d42d49fba16c,MMRF
991,9046fcfb-0e05-48fc-a311-d6ef2a422ae1,MMRF
992,56ed99a7-42bb-46a1-acd9-c66733fece55,MMRF
993,de405770-7784-4ca2-991f-2dee14ef221b,MMRF


###test 5 - duplication verifcation

**5. Check for any duplicate rows present in the table**

In [91]:
%%bigquery --project isb-project-zero

SELECT count(case_id) AS count
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_follow`
group by follow__follow_up_id, case_id, follow__mol_test__count, follow__days_to_follow_up, follow__height, follow__weight, follow__ecog_performance_status, follow__state, follow__created_datetime, follow__updated_datetime
ORDER BY count DESC
LIMIT 10

,count
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


###test 6 - case_id master clinical data table count verifcation

**6. Verify case_id count of table against master rel_clinical_data table**

In [76]:
# case_id count from the program MMRF clinical table

%%bigquery --project isb-project-zero

select distinct case_id, count(case_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF` 
group by case_id
order by count

,case_id,count
0,e8a89081-0ada-4b93-9bf9-399597ea9d76,1
1,1365c987-1c56-474e-ab2b-7f53cabf3cb0,1
2,6b9d857e-e76b-4901-8959-506801628eaf,1
3,463a106f-de45-4520-8122-aaed112641bb,1
4,a3b3a7dd-8559-4b01-8112-d15ec35c5550,1
...,...,...
990,c93b9815-3f76-48c6-8b6c-1ec2ee08825d,1
991,ddb370db-7348-49a2-bcb0-5afdefa6d644,1
992,851f305b-e03c-4648-9400-facaa7123cc6,1
993,2ff5c839-e056-4fcc-a4a5-43f11535024e,1


In [77]:
# case_id count from the program MMRF_follow clinical table

%%bigquery --project isb-project-zero

select distinct case_id, count(case_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_follow` 
group by case_id
order by count

,case_id,count
0,8b86cd08-9096-4ba6-9167-e54a77e356b0,1
1,15713a83-8a83-491e-9b6d-52538a3e8478,1
2,1eb5a034-f107-42f8-ac42-b0fe57514701,1
3,8db0ed0d-03bb-4bc6-917d-4bd861472ed1,2
4,b1295fc9-a0ce-4e60-b6d5-a28234c5b3e4,2
...,...,...
990,8ed77787-c930-42b7-90e5-4267aa6f9854,21
991,1778f74c-366b-48bb-9528-9ac10378389f,21
992,71d62f8d-51db-4b5b-b3d3-a364b70f70a7,22
993,07d7c87a-409f-437d-bef8-36511773d540,22


##Clin MMRF_follow__mol_test

**Testing Full ID** `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_follow__mol_test`

[Table location](https://console.cloud.google.com/bigquery?project=high-transit-276919&authuser=2&p=isb-project-zero&d=GDC_metadata&t=rel23_fileData_slide2caseIDmap&page=table)

Source : GDC API

Release version : v24

###test 1 - schema verification

**1. Check schema**

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields
    
Are the labels correct

Google documentation column descriptions for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#column_field_paths_view).

Google documentation table options for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#options_table).

In [78]:
#return all table information for rel24_clin_MMRF_follow__mol_test

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLES')
clin_query = Query.from_(clin_table) \
                  .select(' table_catalog, table_schema, table_name, table_type ') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_follow__mol_test') \
                  
clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
clin.head()

,table_catalog,table_schema,table_name,table_type
0,isb-project-zero,GDC_Clinical_Data,rel24_clin_MMRF_follow__mol_test,BASE TABLE


In [79]:
#return all table information for rel24_clin_MMRF_follow__mol_test

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_follow__mol_test') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
    print(clin['option_name'][i] + '\n')
    print('\t' + clin['option_value'][i] + '\n')
    print('\t' + clin['option_type'][i] + '\n')

else:

    print('QC of friendly name, table description and labels --- FAILED')

QC of friendly name, table description and labels --- FAILED


In [83]:
#check for empty schemas in dataset rel24_clin_MMRF_follow__mol_test

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.TABLE_OPTIONS')
clin_query = Query.from_(clin_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_follow__mol_test') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows
print("Are there any empty cells in the table schema?")
clin.empty

Are there any empty cells in the table schema?


True

FIELD Descriptions pulled example below


In [82]:
#list of field descriptions for table rel24_clin_MMRF_follow__mol_test

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_follow__mol_test') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(clin)):
  print(clin['table_name'][i] + '\n')
  print('\t' + clin['column_name'][i] + '\n')
  print('\t' + clin['description'][i] + '\n')

rel24_clin_MMRF_follow__mol_test

	follow__mol_test__molecular_test_id

	

rel24_clin_MMRF_follow__mol_test

	follow__follow_up_id

	Reference to ancestor follow__follow_up_id, located in rel24_clin_MMRF_follow.

rel24_clin_MMRF_follow__mol_test

	case_id

	Reference to ancestor case_id, located in rel24_clin_MMRF.

rel24_clin_MMRF_follow__mol_test

	follow__mol_test__biospecimen_type

	The text term used to describe the biological material used for testing, diagnostic, treatment or research purposes.

rel24_clin_MMRF_follow__mol_test

	follow__mol_test__laboratory_test

	The text term used to describe the medical testing used to diagnose, treat or further understand a patient's disease.

rel24_clin_MMRF_follow__mol_test

	follow__mol_test__test_result

	The text term used to describe the result of the molecular test. If the test result was a numeric value see test_value.

rel24_clin_MMRF_follow__mol_test

	follow__mol_test__test_units

	The text term used to describe the units of the 

In [85]:
# check for empty schemas in dataset rel24_clin_MMRF_follow__mol_test

clin_table = Table('`isb-project-zero`.GDC_Clinical_Data.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
clin_query = Query.from_(clin_table) \
                  .select('table_name, column_name, description') \
                  .where(clin_table.table_name=='rel24_clin_MMRF_follow__mol_test') \

clin_query_clean = str(clin_query).replace('"', "")
clin = client.query(clin_query_clean).to_dataframe()
print(clin)

                          table_name  ...                                        description
0   rel24_clin_MMRF_follow__mol_test  ...                                                   
1   rel24_clin_MMRF_follow__mol_test  ...  Reference to ancestor follow__follow_up_id, lo...
2   rel24_clin_MMRF_follow__mol_test  ...  Reference to ancestor case_id, located in rel2...
3   rel24_clin_MMRF_follow__mol_test  ...  The text term used to describe the biological ...
4   rel24_clin_MMRF_follow__mol_test  ...  The text term used to describe the medical tes...
5   rel24_clin_MMRF_follow__mol_test  ...  The text term used to describe the result of t...
6   rel24_clin_MMRF_follow__mol_test  ...  The text term used to describe the units of th...
7   rel24_clin_MMRF_follow__mol_test  ...  The text term or numeric value used to describ...
8   rel24_clin_MMRF_follow__mol_test  ...  The text term used to describe the method used...
9   rel24_clin_MMRF_follow__mol_test  ...  The text term used to descr

###test 2 - row number verification

**2. Look at table row number and size**

Do these metrics make sense?

In [87]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_follow__mol_test`

,f0_
0,134981


In [86]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_id)
FROM `isb-project-zero.GDC_Clinical_Data.rel23_clin_MMRF_follow__mol_test`

,f0_
0,134981


In [88]:
%%bigquery --project isb-project-zero
SELECT *
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_follow__mol_test`

,follow__mol_test__molecular_test_id,follow__follow_up_id,case_id,follow__mol_test__biospecimen_type,follow__mol_test__laboratory_test,follow__mol_test__test_result,follow__mol_test__test_units,follow__mol_test__test_value,follow__mol_test__molecular_analysis_method,follow__mol_test__gene_symbol,follow__mol_test__state,follow__mol_test__created_datetime,follow__mol_test__updated_datetime
0,ebb752ee-a6bb-4d18-8db2-91cc0e6622ad,d4b38843-2de6-4076-832c-2aa7e0e1e6bc,089a1abc-aa81-46a2-b02f-f0f78a1b3d84,Blood,Albumin,Test Value Reported,g/L,46.00000,Not Reported,Not Reported,released,2018-07-23T12:56:17.206927-05:00,2019-06-24T08:07:19.797044-05:00
1,e8353160-439c-45ed-b34b-a5bcab709916,8c70e01c-5eb0-4a11-8b10-2d9a66e4558f,089a1abc-aa81-46a2-b02f-f0f78a1b3d84,Blood,Albumin,Test Value Reported,g/L,47.00000,Not Reported,Not Reported,released,2018-07-23T12:56:17.206927-05:00,2019-06-24T08:07:19.797044-05:00
2,e58b1672-3ef9-42de-b9cb-cca1bf21e98c,c59e0dd2-26f1-47d8-822c-508c2bd61c9a,089a1abc-aa81-46a2-b02f-f0f78a1b3d84,Blood,Albumin,Test Value Reported,g/L,46.00000,Not Reported,Not Reported,released,2018-07-23T12:56:17.206927-05:00,2019-06-24T08:07:19.797044-05:00
3,c3b1044f-dd62-4063-97c9-c4305323e498,a0f90e88-c46c-4ded-9a72-688c2392bd78,089a1abc-aa81-46a2-b02f-f0f78a1b3d84,Blood,Albumin,Test Value Reported,g/L,42.00000,Not Reported,Not Reported,released,2018-07-23T12:56:17.206927-05:00,2019-06-24T08:07:19.797044-05:00
4,0e90f202-af36-4959-b0e5-39357150ee11,961438d6-c5a4-4c50-b38a-9bda1b9fb3cf,089a1abc-aa81-46a2-b02f-f0f78a1b3d84,Blood,Albumin,Test Value Reported,g/L,45.00000,Not Reported,Not Reported,released,2018-07-23T12:56:17.206927-05:00,2019-06-24T08:07:19.797044-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134976,b4a22905-ff7a-4ed2-a4c8-4bebf46434ae,aed502d5-4e1f-4cf5-8f5e-e8f1e2a4ffc5,9c7c0020-7cd3-4e0f-86cd-b474f82fb732,None,Lactate Dehydrogenase,Test Value Reported,ukat/L,3.10062,Not Reported,Not Reported,released,2018-10-09T02:32:37.314168-05:00,2019-06-24T08:07:19.797044-05:00
134977,5d3be2d2-c0b9-4456-897b-317920376e02,d4ac553c-93e0-456d-b03b-b957addffdac,9c7c0020-7cd3-4e0f-86cd-b474f82fb732,None,Lactate Dehydrogenase,Test Value Reported,ukat/L,3.05061,Not Reported,Not Reported,released,2018-10-09T02:32:37.314168-05:00,2019-06-24T08:07:19.797044-05:00
134978,8fae9117-0705-407a-8c90-d95108855ce8,5b9e5c25-3c95-4a0b-b69a-4a9fea0426d0,9c7c0020-7cd3-4e0f-86cd-b474f82fb732,None,Lactate Dehydrogenase,Test Value Reported,ukat/L,5.31773,Not Reported,Not Reported,released,2018-10-09T02:32:37.314168-05:00,2019-06-24T08:07:19.797044-05:00
134979,897a446d-0da8-41a0-aaf7-aa59cd525af6,2166d87f-becb-47c1-aa92-6ea85b9f9b26,9c7c0020-7cd3-4e0f-86cd-b474f82fb732,None,Lactate Dehydrogenase,Test Value Reported,ukat/L,4.55091,Not Reported,Not Reported,released,2018-10-09T02:32:37.314168-05:00,2019-06-24T08:07:19.797044-05:00


###test 3 - manual verification

**3. Scroll through table manually**

See if anything stands out - empty columns, etc.

The BigQuery table search user interface is useful in for this test run. The test tier points to the isb-etl-open. 

ISB-CGC BigQuery table  search [test tier](https://isb-cgc-test.appspot.com/bq_meta_search/).

BigQuery console [isb-project-zero](https://console.cloud.google.com/bigquery?project=high-transit-276919&authuser=2&p=isb-project-zero&d=GDC_metadata&t=rel24_clin_MMRF_follow__mol_test&page=table).

Run a manual check in the console with the steps mentioned in step 1 

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields?
    
Are the labels correct?

###test 4 - case_gdc_id file metadata table count verification

**4. Number of case_id versus BigQuery metadata table**

In [89]:
# clinical case_id counts table reuslts below

# Query below will display the number of cases presents in this table.

clin_table = Table('`isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_follow__mol_test`')
clin_query = Query.from_(clin_table) \
                  .select(' DISTINCT case_id, count(*) as count') \
                  .groupby('case_id')

clin_query_clean = str(clin_query).replace('"', "")
#print(clin_query_clean)
clin = client.query(clin_query_clean).to_dataframe()
print('number of case from submitter_id = ' + str(len(clin.index)))

number of case from submitter_id = 995


In [90]:
# GDC file metadata table case_gdc_id count for clinical below

%%bigquery --project isb-project-zero
SELECT case_gdc_id, program_name
FROM `isb-project-zero.GDC_metadata.rel24_caseData`
where program_name = 'MMRF'
group by case_gdc_id, program_name

,case_gdc_id,program_name
0,a6533b38-fe7d-4c1c-951c-c868e9377af6,MMRF
1,f1b12b03-6135-40a1-9aef-0a600e3b5b9a,MMRF
2,b4f423b6-71b9-459a-a5ed-93208ec27397,MMRF
3,9a2b0af5-bcf5-4398-bcd0-d4f4c7cc9852,MMRF
4,e5992503-81fd-4ba2-acbb-33901a9d498a,MMRF
...,...,...
990,6357d58b-499d-48ac-9195-d42d49fba16c,MMRF
991,9046fcfb-0e05-48fc-a311-d6ef2a422ae1,MMRF
992,56ed99a7-42bb-46a1-acd9-c66733fece55,MMRF
993,de405770-7784-4ca2-991f-2dee14ef221b,MMRF


###test 5 - duplication verifcation

**5. Check for any duplicate rows present in the table**

In [92]:
%%bigquery --project isb-project-zero

SELECT count(case_id) AS count
FROM `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_follow__mol_test`
group by follow__mol_test__molecular_test_id, follow__follow_up_id, case_id, follow__mol_test__biospecimen_type, follow__mol_test__laboratory_test, follow__mol_test__test_result, follow__mol_test__test_units, follow__mol_test__test_value, follow__mol_test__molecular_analysis_method, follow__mol_test__gene_symbol, follow__mol_test__state, follow__mol_test__created_datetime, follow__mol_test__updated_datetime 
ORDER BY count DESC
LIMIT 10

,count
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


###test 6 - case_id master clinical data table count verifcation

**6. Verify case_id count of table against master rel_clinical_data table**

In [93]:
# case_id count from the program MMRF clinical table

%%bigquery --project isb-project-zero

select distinct case_id, count(case_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF` 
group by case_id
order by count

,case_id,count
0,e8a89081-0ada-4b93-9bf9-399597ea9d76,1
1,1365c987-1c56-474e-ab2b-7f53cabf3cb0,1
2,6b9d857e-e76b-4901-8959-506801628eaf,1
3,463a106f-de45-4520-8122-aaed112641bb,1
4,a3b3a7dd-8559-4b01-8112-d15ec35c5550,1
...,...,...
990,c93b9815-3f76-48c6-8b6c-1ec2ee08825d,1
991,ddb370db-7348-49a2-bcb0-5afdefa6d644,1
992,851f305b-e03c-4648-9400-facaa7123cc6,1
993,2ff5c839-e056-4fcc-a4a5-43f11535024e,1


In [94]:
# case_id count from the program MMRF_follow clinical table

%%bigquery --project isb-project-zero

select distinct case_id, count(case_id) as count
from `isb-project-zero.GDC_Clinical_Data.rel24_clin_MMRF_follow__mol_test` 
group by case_id
order by count

,case_id,count
0,4a4e9116-4e5f-4f8b-8f7d-ae9f3389e8a6,8
1,4c216c0a-7d6f-47f3-8c81-1693cd3abfdf,8
2,7e7e4059-d620-4c8f-93e5-617b971290dc,8
3,74d6608a-e201-488c-aab9-cea481ca45b8,8
4,ad99e7b0-9222-4c58-8dac-141eaf45ad46,9
...,...,...
990,d6742460-d3b9-41f0-98be-f117a8d1609c,351
991,a3b3a7dd-8559-4b01-8112-d15ec35c5550,354
992,b9071bd7-8172-4eb2-8192-b28d4d9233a8,358
993,1778f74c-366b-48bb-9528-9ac10378389f,360
